# SELECT within SELECT

In [1]:
import getpass
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
pwd = getpass.getpass()
engine = create_engine(
    'postgresql+psycopg2://postgres:%s@localhost/sqlzoo' % (pwd))
pd.set_option('display.max_rows', 100)

 ····


## 1. Bigger than Russia

**List each country name where the population is larger than that of 'Russia'.**

```
world(name, continent, area, population, gdp)
```

In [2]:
world = pd.read_sql_table('world', engine)

In [3]:
bm = world.loc[world['name']=='Russia', 'population'].values[0]
world.loc[world['population'] > bm, ['name']]

,name
13,Bangladesh
23,Brazil
35,China
76,India
77,Indonesia
125,Nigeria
130,Pakistan
185,United States


## 2. Richer than UK

**Show the countries in Europe with a per capita GDP greater than 'United Kingdom'.**

> _Per Capita GDP_   
> The per capita GDP is the gdp/population

In [4]:
world['pcgdp'] = world['gdp']/world['population']
bm = world.loc[world['name']=='United Kingdom', 'pcgdp'].values[0]
world.loc[(world['pcgdp']>bm) & (world['continent']=='Europe'), ['name']]

,name
3,Andorra
9,Austria
16,Belgium
46,Denmark
59,Finland
64,Germany
75,Iceland
80,Ireland
97,Liechtenstein
99,Luxembourg


## 3. Neighbours of Argentina and Australia


List the name and continent of countries in the continents containing either Argentina or Australia. Order by name of the country.

In [5]:
world.loc[world['continent'].isin(
    world.loc[world['name'].isin(['Argentina', 'Australia']), 'continent']), 
          ['name', 'continent']]

,name,continent
6,Argentina,South America
8,Australia,Oceania
20,Bolivia,South America
23,Brazil,South America
34,Chile,South America
36,Colombia,South America
50,Ecuador,South America
58,Fiji,Oceania
71,Guyana,South America
88,Kiribati,Oceania


## 4. Between Canada and Poland

Which country has a population that is more than Canada but less than Poland? Show the name and the population.

In [6]:
world.loc[(world['population']>
    world.loc[world['name']=='Canada', 'population'].values[0]) &
    (world['population']<world.loc[world['name']=='Poland', 'population'].values[0]), 
          ['name', 'population']]

,name,population


## 5. Percentages of Germany

Germany (population 80 million) has the largest population of the countries in Europe. Austria (population 8.5 million) has 11% of the population of Germany.

**Show the name and the population of each country in Europe. Show the population as a percentage of the population of Germany.**

The format should be Name, Percentage for example:

name	| percentage
--------|-----------
Albania	| 3%
Andorra	| 0%
Austria	| 11%
...	| ...

> _Decimal places_   
> You can use the function ROUND to remove the decimal places.

> _Percent symbol %_
> You can use the function CONCAT to add the percentage symbol.

[To get a well rounded view of the important features of SQL you should move on to the next tutorial concerning aggregates.](https://sqlzoo.net/wiki/SUM_and_COUNT)

To gain an absurdly detailed view of one insignificant feature of the language, read on.

We can use the word `ALL` to allow >= or > or < or <=to act over a list. For example, you can find the largest country in the world, by population with this query:

```sql
SELECT name
  FROM world
 WHERE population >= ALL(SELECT population
                           FROM world
                          WHERE population>0)
```

You need the condition **population>0** in the sub-query as some countries have **null** for population.

In [7]:
bm = world.loc[world['name']=='Germany', ['population']].values[0]
(world.loc[world['continent']=='Europe', :]
    .assign(pct=round(100*world['population']/bm, 0).astype(int).astype(str)+'%')
    .loc[:, ['name', 'pct']])

,name,pct
1,Albania,3%
3,Andorra,0%
9,Austria,11%
15,Belarus,11%
16,Belgium,14%
21,Bosnia and Herzegovina,4%
25,Bulgaria,8%
42,Croatia,5%
45,Czech Republic,13%
46,Denmark,7%


## 6. Bigger than every country in Europe

Which countries have a GDP greater than every country in Europe? [Give the name only.] (Some countries may have NULL gdp values)

We can refer to values in the outer SELECT within the inner SELECT. We can name the tables so that we can tell the difference between the inner and outer versions.

In [8]:
bm = world.loc[world['continent']=='Europe', 'gdp'].max()
world.loc[world['gdp']>bm, ['name']]

,name
35,China
84,Japan
185,United States


## 7. Largest in each continent

**Find the largest country (by area) in each continent, show the continent, the name and the area:**

```sql
SELECT continent, name, population FROM world x
  WHERE population >= ALL
    (SELECT population FROM world y
        WHERE y.continent=x.continent
          AND population>0)
```

> __The above example is known as a correlated or synchronized sub-query.__   
> 
> Using correlated subqueries
> A correlated subquery works like a nested loop: the subquery only has access to rows related to a single > record at a time in the outer query. The technique relies on table aliases to identify two different uses of the same table, one in the outer query and the other in the subquery.
> 
> One way to interpret the line in the **WHERE** clause that references the two table is _“… where the correlated values are the same”._
> 
> In the example provided, you would say _“select the country details from world where the population is greater than or equal to the population of all countries where the continent is the same”._

In [9]:
idx = world.groupby('continent')['area'].transform(max)==world['area']
pd.DataFrame(world.loc[idx, ['continent', 'name', 'area']])

,continent,name,area
2,Africa,Algeria,2381741.0
8,Oceania,Australia,7692024.0
23,South America,Brazil,8515767.0
31,North America,Canada,9984670.0
35,Asia,China,9596961.0
43,Caribbean,Cuba,109884.0
86,Europe,Kazakhstan,2724900.0
110,"Federated States of,Oceania",Micronesia,702.0
141,Eurasia,Russia,17125242.0


## 8. First country of each continent (alphabetically)

**List each continent and the name of the country that comes first alphabetically.**

In [10]:
idx = world.groupby('continent')['name'].transform(min)==world['name']
pd.DataFrame(world.loc[idx, ['continent', 'name']])

,continent,name
0,Asia,Afghanistan
1,Europe,Albania
2,Africa,Algeria
5,Caribbean,Antigua and Barbuda
6,South America,Argentina
7,Eurasia,Armenia
8,Oceania,Australia
17,North America,Belize
110,"Federated States of,Oceania",Micronesia


## 9. Difficult Questions That Utilize Techniques Not Covered In Prior Sections

**Find the continents where all countries have a population <= 25000000. Then find the names of the countries associated with these continents. Show name, continent and population.**

In [11]:
bm = world.loc[world['population']>2.5e7, 'continent']
world.loc[~world['continent'].isin(bm), ['name', 'continent', 'population']]

,name,continent,population
5,Antigua and Barbuda,Caribbean,96453.0
11,Bahamas,Caribbean,385340.0
14,Barbados,Caribbean,287025.0
43,Cuba,Caribbean,11209628.0
48,Dominica,Caribbean,71808.0
49,Dominican Republic,Caribbean,10358320.0
67,Grenada,Caribbean,112003.0
72,Haiti,Caribbean,11577779.0
83,Jamaica,Caribbean,2726667.0
110,Micronesia,"Federated States of,Oceania",101351.0


## 10.

**Some countries have populations more than three times that of any of their neighbours (in the same continent). Give the countries and continents.**

In [12]:
# how many neighbors are there for each continent
bm = pd.DataFrame(world.groupby('continent')['name'].count()-1)
# compare with tripple population
a = world.set_index('continent').join((world
    .assign(popl3=3*world['population'])
    .loc[:,['continent', 'name', 'popl3']]
    .set_index('continent')), on='continent', how='outer', rsuffix='_y')
a = a.loc[a['population'] > a['popl3'], ['name', 'name_y']]
b = a.groupby([a.index, a.name]).count()
b = b.merge(bm, left_index=True, right_index=True)
pd.DataFrame(b.loc[b['name']==b['name_y'], :].index[1])

,0
0,South America
1,Brazil
